In [1]:
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.optimize import fsolve
from Black76 import Black76

In [2]:
df = pd.read_csv('data/data_fx_vol.csv', sep=';')
df.drop(columns=['Unnamed: 2'], inplace=True)
df

,Strike,IV
0,33.882423,1.020475
1,33.977522,1.019650
2,34.072888,1.018825
3,34.168521,1.018000
4,34.264423,1.017174
...,...,...
995,550.972649,0.883735
996,552.519083,0.884475
997,554.069857,0.885214
998,555.624984,0.885952


## ToDo: Rewrite as a class VolSmile or Volsurface.

In [6]:
T=0.520000
F0=124.420000
flag = 'c'
target_delta = 0.75

In [9]:
IV_interp = interp1d(df.Strike, df.IV)

def get_vol_from_delta(
    delta: float
):
    def BS_delta(K, flag):
        return Black76(IV_interp(K)).delta(T, K, F0, flag)

    K_target = fsolve(
        func=lambda x: BS_delta(x, flag) - delta, 
        x0=F0
    )
    return IV_interp(K_target)[0]

def risk_reversal():
    return get_vol_from_delta(0.25) - get_vol_from_delta(0.75)

def butterfly():
    return 0.5 * (get_vol_from_delta(0.25) + get_vol_from_delta(0.75)) - get_vol_from_delta(0.5)

In [16]:
print('Risk reversal =', risk_reversal())
print('Butterfly =', butterfly())

Risk reversal = -0.062030061712471274
Butterfly = 0.03277129851710392
